In [28]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, concatenate, GlobalMaxPooling1D, Dense, LSTM, BatchNormalization, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import LabelEncoder
from datetime import datetime
import os
from pathlib import Path
from sklearn.multioutput import MultiOutputClassifier
from keras.models import Sequential
from sklearn.model_selection import RepeatedKFold
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.metrics import Precision, Recall
from sklearn.metrics import accuracy_score

In [2]:
current_folder_path = os.getcwd()
parent_folder_path = os.path.dirname(current_folder_path)
data_save_path = os.path.join(parent_folder_path, 'train_data')
path = data_save_path + "\\train_data_values_less300_10"
values_pre= pd.read_pickle(path)
path = data_save_path + "\\train_labels_less300_10"
labels_pre= pd.read_pickle(path)

In [3]:
# data_path = "/content/drive/MyDrive/ProjectArbeit/train_data_values_300_10"
# label_path = "/content/drive/MyDrive/ProjectArbeit/train_labels_300_10"
# test_data_path = "/content/drive/MyDrive/ProjectArbeit/test_data_values_300_10"
# test_label_path = "/content/drive/MyDrive/ProjectArbeit/test_labels_300_10"
# model_path = "/content/drive/MyDrive/ProjectArbeit/models"

In [3]:
labels = labels_pre.values
values = values_pre.values


enc = sklearn.preprocessing.OneHotEncoder(categories='auto')
enc.fit(np.concatenate((labels), axis=0).reshape(-1, 1))
labels = enc.transform(labels.reshape(-1, 1)).toarray()

# input_shape = values.shape[1:]
# num_classes = labels.shape[1]

In [5]:
labels_generated = pd.read_pickle('mixed_label_data')
values_generated = pd.read_pickle('mixed_train_data')

In [16]:
labels_generated = labels_generated.values

original_array = labels_generated

num_columns = original_array.shape[1]

identity_matrix = np.eye(num_columns, dtype=np.float64)

transformed_array = original_array.dot(identity_matrix)

labels = np.concatenate((labels,transformed_array),axis=0 )
values_generated = values_generated.values
values = np.concatenate((values,values_generated), axis = 0)

In [27]:
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
for train_ix, test_ix in cv.split(values):
    # prepare data
    X_train, X_test = values[train_ix], values[test_ix]
    y_train, y_test = labels[train_ix], labels[test_ix]

    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

In [32]:
input_shape = X_train.shape[1:]
num_classes = len(np.unique(np.concatenate((y_train, y_test), axis=0)))

In [33]:
num_classes

6

In [28]:
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (3078, 300, 1)
X_test shape: (341, 300, 1)
y_train shape: (3078, 1)
y_test shape: (341, 1)


In [59]:
# labels = labels_pre.values
# values = values_pre.values
# X_train, X_test, y_train, y_test = train_test_split(values,labels, test_size=0.2, random_state=42)
# X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
# X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# print("X_train shape:", X_train.shape)
# print("X_test shape:", X_test.shape)
# print("y_train shape:", y_train.shape)
# print("y_test shape:", y_test.shape)

# num_classes = len(np.unique(np.concatenate((y_train, y_test), axis=0)))
# input_shape = X_train.shape[1:]

# enc = sklearn.preprocessing.OneHotEncoder(categories='auto')
# enc.fit(np.concatenate((y_train, y_test), axis=0).reshape(-1, 1))
# y_train = enc.transform(y_train.reshape(-1, 1)).toarray()
# y_test = enc.transform(y_test.reshape(-1, 1)).toarray()

X_train shape: (2735, 300, 1)
X_test shape: (684, 300, 1)
y_train shape: (2735, 1)
y_test shape: (684, 1)


In [61]:
input_shape

(300, 1)

In [ ]:
#softmax model
input_layer = Input(shape=input_shape)

conv_branch1 = Conv1D(filters=128, kernel_size=3, activation='relu')(input_layer)

# Convolutional branch
conv_branch = Conv1D(filters=64, kernel_size=3, activation='relu')(conv_branch1)
conv_branch = MaxPooling1D(pool_size=2)(conv_branch)

# First recurrent branch
rnn_branch1 = LSTM(units=64, return_sequences=True)(conv_branch1)

# Second recurrent branch
rnn_branch2 = LSTM(units=64, return_sequences=True)(rnn_branch1)

# Apply global max pooling to the convolutional branch
conv_branch = GlobalMaxPooling1D()(conv_branch)

# Apply global max pooling to the recurrent branches
rnn_branch2 = GlobalMaxPooling1D()(rnn_branch2)

# Concatenate the outputs of all branches
concatenated = concatenate([conv_branch, rnn_branch2])

# Classification layers
dense_layer = Dense(units=128, activation='relu')(concatenated)
output_layer = Dense(units=num_classes, activation='softmax')(dense_layer)

# Create the model
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
tf.keras.utils.plot_model(model,'model.png', show_shapes=True)

In [ ]:
#sigmoid model
input_layer = Input(shape=input_shape)

conv_branch1 = Conv1D(filters=128, kernel_size=3, activation='relu')(input_layer)

# Convolutional branch
conv_branch = Conv1D(filters=64, kernel_size=3, activation='relu')(conv_branch1)
conv_branch = MaxPooling1D(pool_size=2)(conv_branch)

# First recurrent branch
rnn_branch1 = LSTM(units=64, return_sequences=True)(conv_branch1)

# Second recurrent branch
rnn_branch2 = LSTM(units=64, return_sequences=True)(rnn_branch1)

# Apply global max pooling to the convolutional branch
conv_branch = GlobalMaxPooling1D()(conv_branch)

# Apply global max pooling to the recurrent branches
rnn_branch2 = GlobalMaxPooling1D()(rnn_branch2)

# Concatenate the outputs of all branches
concatenated = concatenate([conv_branch, rnn_branch2])

# Classification layers
dense_layer = Dense(units=128, activation='relu')(concatenated)
output_layer = Dense(units=num_classes, activation='sigmoid')(dense_layer)

# Create the model
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
tf.keras.utils.plot_model(model,'model.png', show_shapes=True)

In [102]:
def call_sigmoid_model(input_shape, num_classes):
    #sigmoid model
    input_layer = Input(shape=input_shape)

    conv_branch_in = Conv1D(filters=128, kernel_size=3, activation='relu')(input_layer)

    # Convolutional branch
    conv_branch = Conv1D(filters=128, kernel_size=3, activation='relu')(conv_branch_in)
    conv_branch = MaxPooling1D(pool_size=2)(conv_branch)

    conv_branch1 = Conv1D(filters=64, kernel_size=3, activation='relu')(conv_branch)
    conv_branch1 = MaxPooling1D(pool_size=2)(conv_branch1)

    # First recurrent branch
    rnn_branch1 = LSTM(units=64, return_sequences=True)(conv_branch_in)

    # Second recurrent branch
    rnn_branch2 = LSTM(units=64, return_sequences=True)(rnn_branch1)

    # Apply global max pooling to the convolutional branch
    conv_branch1 = GlobalMaxPooling1D()(conv_branch1)

    # Apply global max pooling to the recurrent branches
    rnn_branch2 = GlobalMaxPooling1D()(rnn_branch2)

    # Concatenate the outputs of all branches
    concatenated = concatenate([conv_branch1, rnn_branch2])

    # Classification layers
    dense_layer = Dense(units=128, activation='relu')(concatenated)
    output_layer = Dense(units=num_classes, activation='sigmoid')(dense_layer)

    # Create the model
    model = Model(inputs=input_layer, outputs=output_layer)

    initial_learning_rate = 0.001

    # Create an instance of the Adam optimizer with the initial learning rate
    optimizer = Adam(learning_rate=initial_learning_rate)

    # Compile the model with the optimizer and additional metrics
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy', Precision(), Recall()])

    # Define a learning rate scheduler callback
    lr_scheduler = ReduceLROnPlateau(factor=0.1, patience=3)

    return model
    # model.summary()
    # tf.keras.utils.plot_model(model,'model.png', show_shapes=True)

In [35]:
def call_sigmoid_model_long(input_shape, num_classes):
    #sigmoid model longer and bigger version
    input_layer = Input(shape=input_shape)
   
    conv_branch_in = Conv1D(filters=256, kernel_size=5, activation='relu')(input_layer)
    conv_branch_in = Conv1D(filters=256, kernel_size=5, activation='relu')(conv_branch_in)
    conv_branch_in = MaxPooling1D(pool_size=2)(conv_branch_in)

    # Convolutional branch
    conv_branch = Conv1D(filters=512, kernel_size=3, activation='relu')(conv_branch_in)
    conv_branch = BatchNormalization()(conv_branch)
    conv_branch = MaxPooling1D(pool_size=2)(conv_branch)

    conv_branch = Conv1D(filters=256, kernel_size=3, activation='relu')(conv_branch)
    conv_branch = BatchNormalization()(conv_branch)
    conv_branch = MaxPooling1D(pool_size=2)(conv_branch)

    conv_branch = Conv1D(filters=256, kernel_size=3, activation='relu')(conv_branch)
    conv_branch = BatchNormalization()(conv_branch)
    conv_branch = MaxPooling1D(pool_size=2)(conv_branch)
    # Recurrent branch
    rnn_branch = LSTM(units=256, return_sequences=True)(conv_branch_in)
    rnn_branch = LSTM(units=256, return_sequences=True)(rnn_branch)
    rnn_branch = LSTM(units=256, return_sequences=True)(rnn_branch)

    # Global max pooling
    conv_branch1 = GlobalMaxPooling1D()(conv_branch)
    rnn_branch2 = GlobalMaxPooling1D()(rnn_branch)

    # Concatente
    concatenated = concatenate([conv_branch1, rnn_branch2])

    # Output
    dense_layer = Dense(units=128, activation='relu')(concatenated)
    dense_layer = Dropout(0.5)(dense_layer)
    output_layer = Dense(units=num_classes, activation='sigmoid')(dense_layer)


    model = Model(inputs=input_layer, outputs=output_layer)

    initial_learning_rate = 0.001

    optimizer = Adam(learning_rate=initial_learning_rate)

    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy', Precision(), Recall()])


    return model
    # model.summary()
    # tf.keras.utils.plot_model(model,'model.png', show_shapes=True)

In [36]:
model = call_sigmoid_model_long((300, 1),4)
model.summary()
tf.keras.utils.plot_model(model,'model.png', show_shapes=True)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 300, 1)]     0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 296, 256)     1536        ['input_1[0][0]']                
                                                                                                  
 conv1d_1 (Conv1D)              (None, 292, 256)     327936      ['conv1d[0][0]']                 
                                                                                                  
 max_pooling1d (MaxPooling1D)   (None, 146, 256)     0           ['conv1d_1[0][0]']               
                                                                                              

In [ ]:
# define the model
model = Sequential()
model.add(Dense(20, input_dim=300, kernel_initializer='he_uniform', activation='relu'))
model.add(Dense(num_classes, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam')

In [38]:
# data_path = "/content/drive/MyDrive/ProjectArbeit/train_data_values_less300_10"
# label_path = "/content/drive/MyDrive/ProjectArbeit/train_labels_less300_10"
# test_data_path = "/content/drive/MyDrive/ProjectArbeit/test_data_values_less300_10"
# test_label_path = "/content/drive/MyDrive/ProjectArbeit/test_labels_less300_10"
# model_path = "/content/drive/MyDrive/ProjectArbeit/models"

# current_folder_path = os.getcwd()
# parent_folder_path = os.path.dirname(current_folder_path)
# data_save_path = os.path.join(parent_folder_path, 'train_data')
# path = data_save_path + "\\train_data_values_less300_10"
# values_pre= pd.read_pickle(path)
# path = data_save_path + "\\train_labels_less300_10"
# labels_pre= pd.read_pickle(path)

# # values_pre= pd.read_pickle(data_path)
# # labels_pre= pd.read_pickle(label_path)

# labels = labels_pre.values
# values = values_pre.values

# enc = sklearn.preprocessing.OneHotEncoder(categories='auto')
# enc.fit(np.concatenate((labels), axis=0).reshape(-1, 1))
# labels = enc.transform(labels.reshape(-1, 1)).toarray()

X = values
y = labels
batch_size = 32
nb_epochs = 15
n_inputs, n_outputs = (300, 1) , 4
results = list()
lr_scheduler = ReduceLROnPlateau(factor=0.1, patience=10, monitor='loss')
# define evaluation procedure
cv = RepeatedKFold(n_splits=10, n_repeats=1, random_state=1)

for train_ix, test_ix in cv.split(values):
    # prepare data
    X_train, X_test = X[train_ix], X[test_ix]
    y_train, y_test = y[train_ix], y[test_ix]

    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

    print("X_train shape:", X_train.shape)
    print("X_test shape:", X_test.shape)
    print("y_train shape:", y_train.shape)
    print("y_test shape:", y_test.shape)
    # define model
    model = call_sigmoid_model_long(n_inputs, n_outputs)
    # fit model
    history = model.fit(X_train, y_train, batch_size=batch_size, epochs=nb_epochs, callbacks=[lr_scheduler])
    # make a prediction on the test set
    yhat = model.predict(X_test)
    # round probabilities to class labels
    yhat = yhat.round()
    # calculate accuracy
    acc = accuracy_score(y_test, yhat)
    # store result
    print('>%.3f' % acc)
    results.append(acc)


print('Accuracy: %.3f (%.3f)' % (np.mean(results), np.std(results)))
modelName= 'Sigmoid_long'
model_path = r"C:\Users\UTKU\OneDrive\Desktop\MS-Term4\ProjectArbeit\models"
now = datetime.now()
timestamp = now.strftime("%d-%m-%H-%M")
model_filename = model_path + '/' + modelName + f"_{timestamp}.h5"
model.save(model_filename)
print(f"Model saved as {model_filename}")


X_train shape: (6156, 300, 1)
X_test shape: (684, 300, 1)
y_train shape: (6156, 4)
y_test shape: (684, 4)
Epoch 1/15
 66/193 [=========>....................] - ETA: 4:11 - loss: 0.5142 - accuracy: 0.5156 - precision_1: 0.7055 - recall_1: 0.6484

KeyboardInterrupt: 

In [93]:
values_pre.to_excel("300_less_value.xlsx")

In [17]:
print('Accuracy: %.3f (%.3f)' % (np.mean(results), np.std(results)))
modelName= 'Sigmoid_with_RKFold_and_lr'
model_path = "C:/Users/UTKU/OneDrive/Desktop/MS-Term4/ProjectArbeit/models"
now = datetime.now()
timestamp = now.strftime("%d-%m-%H-%M")
model_filename = model_path + '/' + modelName + f"_{timestamp}.h5"
model.save(model_filename)
print(f"Model saved as {model_filename}")

Accuracy: 0.990 (0.008)
Model saved as C:/Users/UTKU/OneDrive/Desktop/MS-Term4/ProjectArbeit/models/Sigmoid_with_RKFold_and_lr_15-07-23-54.h5


In [94]:
results

[0.9963503649635036,
 0.9890510948905109,
 1.0,
 0.9744525547445255,
 0.9963503649635036,
 0.9817518248175182,
 0.9963369963369964,
 0.9853479853479854,
 0.9963369963369964,
 0.9853479853479854]

In [24]:
modelName= 'Sigmoid_with_RKFold_and_lr'
now = datetime.now()
timestamp = now.strftime("%d-%m-%H-%M")
model_filename = model_path + '/' + modelName + f"_{timestamp}.h5"
model.save(model_filename)
print(f"Model saved as {model_filename}")

Model saved as /content/drive/MyDrive/ProjectArbeit/models/Sigmoid_with_RKFold_and_lr_14-07-14-03.h5


In [38]:
from tensorflow.keras.callbacks import EarlyStopping
earlystopping = False
lr_scheduler = True
modelName= 'Model'

batch_size = 32
nb_epochs = 500

if earlystopping:
    # Define early stopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=75, restore_best_weights=True)

    # Compile the model
    #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Train the model with optimizations
    history = model.fit(X_train, y_train, batch_size=batch_size, epochs=nb_epochs,
                    validation_data=(X_test, y_test), callbacks=[early_stopping])

else:
   history = model.fit(X_train, y_train, batch_size=batch_size, epochs=nb_epochs,
                    validation_data=(X_test, y_test))


now = datetime.now()
timestamp = now.strftime("%d-%m-%H-%M")
model_filename = model_path + '/' + modelName + f"_{timestamp}.h5"
model.save(model_filename)
print(f"Model saved as {model_filename}")


Epoch 1/500


ValueError: ignored

In [ ]:
model_path1 = model_path + '/' + 'P_RNN_CNN_sigmoid3__10-07-22-21.h5'
model = tf.keras.models.load_model(model_path1)

In [15]:
visualize(history, 199, modelName)

KeyError: 'val_accuracy'

In [ ]:
now = datetime.now()
timestamp = now.strftime("%d-%m-%H-%M")
model_filename = model_path + '/' + modelName + '_' + f"_{timestamp}.h5"
model.save(model_filename)
print(f"Model saved as {model_filename}")

Model saved as /content/drive/MyDrive/ProjectArbeit/models/P_RNN_CNN_sigmoid2__10-07-19-02.h5


In [18]:

test_data_path = "C:/Users/UTKU/OneDrive/Desktop/MS-Term4/ProjectArbeit/test_data/test_data_values_less300_10"
test_label_path = "C:/Users/UTKU/OneDrive/Desktop/MS-Term4/ProjectArbeit/test_data/test_data_labels_less300_10"

test_values = pd.read_pickle(test_data_path)
test_labels = pd.read_pickle(test_label_path)

test_values = test_values.values
test_values = test_values.reshape((test_values.shape[0], test_values.shape[1], 1))

predictions = model.predict(test_values)

64/64 [==============================] - 7s 113ms/step


In [23]:
test_label_path = "C:/Users/UTKU/OneDrive/Desktop/MS-Term4/ProjectArbeit/test_data/test_labels_less300_10"

test_labels = pd.read_pickle(test_label_path)
test_labels.to_excel('test_data_labels_less300_10.xlsx')

In [27]:
predictions

array([[3.6781836e-01, 5.7966161e-01, 1.3927696e-09, 3.1160231e-04],
       [4.0128991e-02, 9.8508090e-01, 4.7613300e-09, 4.7525125e-05],
       [4.8066914e-04, 9.9985886e-01, 4.6288317e-08, 1.1577538e-05],
       ...,
       [9.2849914e-05, 1.9537350e-03, 3.0941242e-01, 6.8775696e-01],
       [2.6745041e-05, 8.8108925e-04, 5.7322556e-01, 4.4810566e-01],
       [1.0577276e-04, 2.1574064e-03, 2.8284380e-01, 7.0976216e-01]],
      dtype=float32)

In [92]:
prediction_list = []

threshold = 0.3
# Iterate over each row of predictions
for row in predictions:

    sorted_indices = np.argsort(row)[::-1]
    prediction1 = sorted_indices[0]

    if row[sorted_indices[1]] >= threshold:
        prediction2 = sorted_indices[1]
    else:
        prediction2 = 4

    prediction_list.append([prediction1, prediction2])

prediction_list = np.array(prediction_list)

def convert_array_to_labels(array):
    labels1 = ['Bias','Drift','Gain','Precisiondegradation','NoFault']
    converted_array = [labels1[i] for i in array]
    return converted_array

predicted_labels_1 = convert_array_to_labels(prediction_list[:, 0])
predicted_labels_2 = convert_array_to_labels(prediction_list[:, 1])

# label_encoder = LabelEncoder()
# label_encoder.fit(labels1)
# predicted_labels_1 = label_encoder.inverse_transform(prediction_list[:, 0])
# predicted_labels_2 = label_encoder.inverse_transform(prediction_list[:, 1])
df_predictions = pd.DataFrame({'Prediction': predicted_labels_1, 'Prediction2': predicted_labels_2})

now = datetime.now()
timestamp = now.strftime("%d-%m-%H-%M")
prediction_name = model_path + '/' + modelName + f"_{timestamp}" + f"_{threshold}_" + "_predictions.xlsx"
print(df_predictions)
df_predictions.to_excel(prediction_name)

                Prediction           Prediction2
0                    Drift                  Bias
1                    Drift               NoFault
2                    Drift               NoFault
3                    Drift               NoFault
4                    Drift               NoFault
...                    ...                   ...
2041  Precisiondegradation               NoFault
2042  Precisiondegradation               NoFault
2043  Precisiondegradation                  Gain
2044                  Gain  Precisiondegradation
2045  Precisiondegradation               NoFault

[2046 rows x 2 columns]


In [82]:
current_folder_path = os.getcwd()
parent_folder_path = os.path.dirname(current_folder_path)
data_save_path = os.path.join(parent_folder_path, 'train_data')
path = data_save_path + "\\train_data_values_less300_10"
values_pre= pd.read_pickle(path)
path = data_save_path + "\\train_labels_less300_10"
labels_pre= pd.read_pickle(path)

# values_pre= pd.read_pickle(data_path)
# labels_pre= pd.read_pickle(label_path)

labels = labels_pre.values
values = values_pre.values

enc = sklearn.preprocessing.OneHotEncoder(categories='auto')
enc.fit(np.concatenate((labels), axis=0).reshape(-1, 1))
labels = enc.transform(labels.reshape(-1, 1)).toarray()

In [66]:
prediction_list[:, 1]

array([0, 4, 4, ..., 2, 3, 2], dtype=int64)

In [90]:
line = 2045
print(predictions[line])
print(prediction_list[line])
print(df_predictions.iloc[line])
print(predicted_labels_1[line])
print(predicted_labels_2[line])

[1.0577276e-04 2.1574064e-03 2.8284380e-01 7.0976216e-01]
[3 2]
Prediction     Precisiondegradation
Prediction2                    Gain
Name: 2045, dtype: object
Precisiondegradation
Gain


In [37]:
print(predicted_labels_2[716])

Precisiondegradation


In [14]:
def visualize(history, epochs, title):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs_range = range(epochs)

    plt.figure(figsize=(8, 8))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title(title + ' Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title(title + ' Loss')
    plt.show()

In [42]:
# mlp for multi-label classification
from numpy import mean
from numpy import std
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score

# get the dataset
def get_dataset():
	X, y = make_multilabel_classification(n_samples=1000, n_features=10, n_classes=3, n_labels=2, random_state=1)
	return X, y

# get the model
def get_model(n_inputs, n_outputs):
	model = Sequential()
	model.add(Dense(20, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
	model.add(Dense(n_outputs, activation='sigmoid'))
	model.compile(loss='binary_crossentropy', optimizer='adam')
	return model

# evaluate a model using repeated k-fold cross-validation
def evaluate_model(X, y):
	results = list()
	n_inputs, n_outputs = X.shape[1], y.shape[1]
	# define evaluation procedure
	cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
	# enumerate folds
	for train_ix, test_ix in cv.split(X):
		# prepare data
		X_train, X_test = X[train_ix], X[test_ix]
		y_train, y_test = y[train_ix], y[test_ix]
		# define model
		model = get_model(n_inputs, n_outputs)
		# fit model
		model.fit(X_train, y_train, verbose=0, epochs=100)
		# make a prediction on the test set
		yhat = model.predict(X_test)
		# round probabilities to class labels
		yhat = yhat.round()
		# calculate accuracy
		acc = accuracy_score(y_test, yhat)
		# store result
		print('>%.3f' % acc)
		results.append(acc)
	return results

# load dataset
X, y = get_dataset()
# # evaluate model
# results = evaluate_model(X, y)
# # summarize performance
# print('Accuracy: %.3f (%.3f)' % (mean(results), std(results)))

In [44]:
y

array([[1, 1, 0],
       [0, 0, 0],
       [1, 1, 0],
       ...,
       [1, 1, 0],
       [1, 1, 0],
       [1, 1, 1]])

In [ ]:
y_train1 = y_train[:][0]


In [ ]:
y_train1

array([3])

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(multi_class='ovr')
# fit model
model.fit(X_train, y_train1)
# make predictions
yhat = model.predict(X_test)

In [ ]:
y_test1 = y_test.flatten()

In [ ]:
# Calculate element-wise similarity
similarity = np.sum(y_test1 == yhat)

591/len(yhat) *100
# Print the similarity value
print(f"The element-wise similarity between the arrays is: {similarity}")

The element-wise similarity between the arrays is: 591


In [ ]:
similarity/len(yhat) *100

86.40350877192982

In [ ]:
yhat

array([3, 6, 1, 4, 2, 2, 3, 1, 6, 3, 3, 4, 1, 2, 1, 4, 6, 3, 4, 4, 6, 6,
       2, 4, 1, 3, 3, 3, 4, 4, 2, 6, 4, 1, 1, 1, 2, 4, 3, 1, 2, 2, 4, 3,
       4, 4, 4, 3, 3, 2, 2, 3, 1, 4, 2, 2, 1, 3, 4, 6, 4, 3, 1, 2, 2, 4,
       4, 4, 2, 6, 4, 1, 1, 2, 3, 4, 1, 4, 3, 6, 3, 6, 3, 2, 2, 4, 1, 3,
       3, 6, 6, 1, 3, 4, 2, 2, 3, 4, 4, 1, 4, 2, 4, 1, 1, 1, 4, 6, 4, 1,
       2, 4, 4, 6, 1, 6, 2, 1, 1, 3, 4, 1, 6, 3, 1, 2, 4, 2, 2, 2, 4, 3,
       6, 3, 1, 4, 3, 1, 1, 3, 1, 4, 4, 6, 4, 1, 2, 1, 2, 1, 4, 6, 3, 3,
       4, 4, 4, 4, 4, 4, 1, 6, 4, 3, 1, 2, 3, 3, 4, 6, 2, 3, 1, 1, 2, 1,
       1, 4, 1, 1, 3, 6, 4, 4, 2, 4, 2, 3, 1, 1, 3, 2, 3, 2, 2, 2, 4, 4,
       2, 4, 2, 4, 1, 2, 4, 4, 2, 6, 2, 3, 3, 1, 4, 1, 3, 3, 1, 1, 2, 1,
       4, 4, 6, 3, 1, 1, 1, 3, 6, 4, 2, 1, 6, 1, 3, 4, 4, 4, 4, 1, 3, 3,
       3, 4, 2, 6, 4, 1, 1, 1, 2, 1, 4, 2, 4, 3, 4, 3, 4, 3, 3, 3, 4, 4,
       3, 2, 2, 3, 3, 1, 3, 4, 4, 3, 2, 4, 3, 4, 1, 6, 4, 4, 2, 2, 2, 4,
       3, 1, 1, 4, 2, 4, 4, 1, 2, 4, 3, 1, 3, 2, 3,